In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import requests
import yaml

import torch
import matplotlib.pyplot as plt
%matplotlib inline

import tiktoken
enc = tiktoken.encoding_for_model('text-davinci-001')
def count_tokens(s): return len(enc.encode(s))

## Spotify api spec

In [3]:
# https://github.com/APIs-guru/openapi-directory
!wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml

--2023-03-27 15:30:47--  https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286747 (280K) [text/plain]
Saving to: ‘openapi.yaml’

openapi.yaml        100%[===================>] 280.03K  --.-KB/s    in 0.06s   

2023-03-27 15:30:48 (4.65 MB/s) - ‘openapi.yaml’ saved [286747/286747]



In [4]:
with open('openapi.yaml', 'r') as f:
    spotify_api_spec = yaml.load(f, Loader=yaml.Loader)    
spotify_api_spec.keys()

dict_keys(['openapi', 'servers', 'info', 'paths', 'components'])

In [5]:
count_tokens(yaml.dump(spotify_api_spec))

137242

In [6]:
endpoints = [
    (route, operation)
    for route, operations in spotify_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

63

In [7]:
auth_metadata = spotify_api_spec['components']['securitySchemes']['oauth_2_0']['flows']['authorizationCode']
auth_url = auth_metadata['authorizationUrl']
token_url = auth_metadata['tokenUrl']
scopes = list(auth_metadata['scopes'].keys())
auth_url, token_url

('https://accounts.spotify.com/authorize',
 'https://accounts.spotify.com/api/token')

In [8]:
base_url = spotify_api_spec['servers'][0]['url']
base_url

'https://api.spotify.com/v1'

## Spotify app dev

Setup:
- https://developer.spotify.com/documentation/general/guides/authorization/
- 1) set up an app in Spotify's developer console by hand to source creds (CLIENT_ID, CLIENT_SECRET, REDIRECT_URI)
- 2) implement oauth flows to source access, refresh tokens. alternatively, Spotipy handles this:

In [9]:
# !pip install spotipy
CLIENT_ID = ''
CLIENT_SECRET = ''
REDIRECT_URI = 'http://thisisafakeaddress' # whitelisted

os.environ['SPOTIPY_CLIENT_ID'] = CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = CLIENT_SECRET
os.environ['SPOTIPY_REDIRECT_URI'] = REDIRECT_URI

import spotipy.util as util
access_token = util.prompt_for_user_token(scope=','.join(scopes))

headers = {
    'Authorization': f'Bearer {access_token}'
}

## Spotify agent flows to evaluate... e.g. a dialogue recommender

- https://developer.spotify.com/documentation/web-api/reference/#/operations/get-recommendations
- multi-hop reasoning, cooperative multi-tasking with user
- e.g.
  
```
user: give me some music recs to listen to
agent: <finds recommendation endpoint... finds some seed is required...>
agent: who are some seed artists/songs/etc. i should base this recommedation on?
user: "miles davis"
agent: <hits recommendation endpoint...>
agent: here are some recs
user: ok, play rec[2]
agent: <figures out playback...>
```

In [10]:
artist_name = 'miles davis'
res = requests.get(f'{base_url}/search', params={
    'q': artist_name,
    'type': ['artist']
}, headers=headers)
artist = res.json()['artists']['items'][0]
assert artist['name'].lower() == artist_name
artist['id']

'0kbYTNQb4Pb1rPbbaF0pT4'

In [11]:
res = requests.get(f'{base_url}/recommendations', params={
    'seed_artists': artist['id'],
}, headers=headers)
res_json = res.json()
[track['name'] for track in res_json['tracks']][:5]

['Make the Road By Walking',
 'Come On, Come Over',
 'When I Fall In Love',
 'The Natives Are Restless Tonight - Rudy Van Gelder Edition/1999 Digital Remaster',
 'American Tango']

## Build a structured embedding store of the OAS...

One option is to treat the spec as unstructured, and then apply crude chunking strategies to break up the spec into retrieval units. The other option is to take advantage of the spec structure and embed accordingly. So far, just embedding each endpoint's description has been highest signal, not surpringly.


In [13]:
from langchain.agents.agent_toolkits.openapi import spec as spec_utils

In [14]:
print(count_tokens(yaml.dump(spotify_api_spec['paths']['/recommendations']['get'])))
_spec = spec_utils.reduce_openapi_spec(spotify_api_spec, dereference=True)
print(count_tokens(yaml.dump({name: docs for name, desc, docs in  _spec.endpoints}['GET /recommendations'])))
_spec = spec_utils.reduce_openapi_spec(spotify_api_spec, dereference=False)
print(count_tokens(yaml.dump({name: docs for name, desc, docs in  _spec.endpoints}['GET /recommendations'])))

7035
18195
540


In [15]:
from langchain.embeddings import OpenAIEmbeddings
model_name = 'text-embedding-ada-002'
embedder = OpenAIEmbeddings(model=model_name)

In [16]:
spec = spec_utils.reduce_openapi_spec(spotify_api_spec, dereference=False)

In [17]:
# An API-specific change, but for now just keep a few api resources.
resources = ['albums', 'artists', 'player', 'recommendations', 'search', 'tracks', 'user']
spec.endpoints = [
    (name, description, docs)
    for name, description, docs in spec.endpoints
    if any([resource in name for resource in resources])
]

In [18]:
endpoint_descriptions = [
    f"{name} {description}"
    for name, description, _ in spec.endpoints
]
endpoint_descriptionsv = embedder.embed_documents(endpoint_descriptions)
endpoint_descriptions[:5]

['GET /albums Get Spotify catalog information for multiple albums identified by their Spotify IDs.\n',
 'GET /albums/{id} Get Spotify catalog information for a single album.\n',
 'GET /albums/{id}/tracks Get Spotify catalog information about an album’s tracks.\nOptional parameters can be used to limit the number of tracks returned.\n',
 'GET /artists Get Spotify catalog information for several artists based on their Spotify IDs.\n',
 'GET /artists/{id} Get Spotify catalog information for a single artist identified by their unique Spotify ID.\n']

In [19]:
query = "Who who wrote the song So What?"
queryv = embedder.embed_query(query)

In [20]:
endpoint_descriptionsv = torch.tensor(endpoint_descriptionsv)
queryv = torch.tensor(queryv)
dist = endpoint_descriptionsv @ queryv

In [21]:
# Another API-specific change. API is resource-based and resources must be referenced by ID,
# meaning our agent would have to look up these IDs. This isn't obvious, but a simple cheat would be
# to upweight the search endpoint used for this lookup.
search_idx = None
for idx, text in enumerate(endpoint_descriptions):
    if 'GET /search' in text:
        search_idx = idx
dist[search_idx] *= 1.1

In [22]:
top_k = 5
values, indices = torch.sort(dist, descending=True)
[spec.endpoints[ix][0] for ix in indices][:top_k]

['GET /search',
 'GET /artists/{id}/related-artists',
 'GET /me/tracks',
 'GET /artists/{id}/top-tracks',
 'GET /me/albums']

## Initial agent

In [23]:
from langchain.llms.openai import OpenAI
from langchain.chains.llm import LLMChain
from langchain.agents.mrkl.base import ZeroShotAgent
from langchain.agents.agent import AgentExecutor
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS

#### 1. Retrieval tool (eventually, would switch to langchain.tools.vectorstore.tool.VectorStoreQATool)

In [24]:
from langchain.tools.base import BaseTool

# Oof.
API_SPEC_RETRIEVAL_TOOL_DESCRIPTION = """Can be used to search for relevant API endpoints and the endpoint details.
The input is a short description of desired API functionality.
"""

# Build this out by hand before using existing components e.g. VectorStoreQATool
# 1st: top-k retrieval of endpoint details.
# 2nd: agent-ize: allow LM to 1st retrieve/rank endpoints and 2nd get selected endpoint details..
# ...
class ApiSpecRetrievalTool(BaseTool):
    
    name = 'api_endpoint_retriever'
    description = API_SPEC_RETRIEVAL_TOOL_DESCRIPTION + f"""
    The base url for the API is: {spec.servers[0]['url']}.
    """ 
    _endpoints = spec.endpoints
    _endpoint_descriptionsv = torch.tensor(embedder.embed_documents([
        f"{name} {desc}"
        for name, desc, docs in spec.endpoints
    ]))

    def _run(self, tool_input: str) -> str:
        queryv = torch.tensor(embedder.embed_query(tool_input))
        dist = self._endpoint_descriptionsv @ queryv

        # See comment above...
        search_idx = None
        for idx, text in enumerate([name for name, desc, details in self._endpoints]):
            if 'GET /search' in text:
                search_idx = idx

        dist[search_idx] *= 1.25

        top_k = 5
        top_k_ix = torch.argsort(dist, descending=True)[:top_k]
        
        return yaml.dump({
            self._endpoints[ix][0]: self._endpoints[ix][2]
            for ix in top_k_ix
        })
    
    def _arun(): pass
        

#### 2. Wrap the retrieval tool with another chain or agent, basically acting as a k=1 ranker.

This agent's trajectory is independent from the top-level agent, so won't consume context tokens.

In [25]:
# prompt = PromptTemplate(
#     template="Select the endpoint from the docs that can best be used to answer this query: {query}\n\Docs:{docs}\n\nEndpoint:",
#     input_variables=["query", "docs"],
# )
# LLMChain(
#     OpenAI()
# )

In [26]:
# https://github.com/hwchase17/langchain/blob/7eba828e1b9542de4a0b67d1da2ffcfa83c6baca/langchain/agents/mrkl/base.py#L93-L94
# Prefix -> agent task description
PREFIX="""You are an agent that interacts with an API spec and your task is to select the API endpoint that can best be used to answer a given Question.

Some endpoint takes arguments (a GET route may take url params and a POST route must take body args). Do not answer with an endpoint if its arguments are unknown.
Your final answer should be the name of the endpoint, like GET /random/endpoint.
Do not try to call the endpoint. If you can't find an endpoint that works, say I don't know.
"""
# Suffix -> warm-start the completion.
SUFFIX = """Begin!

Question: {input}
Thought: I can use the endpoint retrieval tool to find a few candidate endpoints. Then I will return the name of one endpoint.
{agent_scratchpad}"""

# describes LM context format i.e. how to construct a ReAct agent trajectory
FORMAT_INSTRUCTIONS

tools = [ApiSpecRetrievalTool()]

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=PREFIX,
    suffix=SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,
    input_variables=None, # defaults to ["input", "agent_scratchpad"]
)

llm_chain = LLMChain(
    llm=OpenAI(model_name='text-davinci-003', temperature=0.0),
    prompt=prompt,
)
tool_names = [tool.name for tool in tools]
endpoint_retrieval_agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [28]:
# return_intermediate_steps=True
# __call__{{'input': query}}
# return_intermediate_steps=True
# run(query: str)
endpoint_retrieval_agent_executor = AgentExecutor.from_agent_and_tools(agent=endpoint_retrieval_agent, tools=tools, verbose=True)

In [29]:
endpoint_retrieval_agent_executor.run("Give me recs for songs like So What")



> Entering new AgentExecutor chain...
Action: api_endpoint_retriever
Action Input: Get recommendations for songs
Observation: GET /artists/{id}/related-artists:
  description: 'Get Spotify catalog information about artists similar to a given artist.
    Similarity is based on analysis of the Spotify community''s [listening history](http://news.spotify.com/se/2010/02/03/related-artists/).

    '
  parameters: []
  responses:
    $ref: '#/components/responses/ManyArtists'
GET /me/albums:
  description: 'Get a list of the albums saved in the current Spotify user''s ''Your
    Music'' library.

    '
  parameters: []
  responses:
    $ref: '#/components/responses/PagingSavedAlbumObject'
GET /me/tracks:
  description: 'Get a list of the songs saved in the current Spotify user''s ''Your
    Music'' library.

    '
  parameters: []
  responses:
    $ref: '#/components/responses/PagingSavedTrackObject'
GET /recommendations:
  description: 'Recommendations are generated based on the available

"I don't know."

#### 3. Wrap it as a tool also.

In [30]:
def retrieval_agent_func(inp: str):
    out = endpoint_retrieval_agent_executor.run(inp)
    return yaml.dump({name: docs for name, _, docs in spec.endpoints}[out])

from langchain.agents.tools import Tool
endpoint_retriever_agent_tool = Tool(
    name="endpoint_selector",
    func=retrieval_agent_func,
    description="""Can be used to find an API endpoint + its documention that will best fulfill a user query."""
)

In [31]:
from langchain.requests import RequestsWrapper
from langchain.tools.requests.tool import (
    RequestsGetTool,
    RequestsPostTool,
)
assert 'Authorization' in headers
requests_wrapper = RequestsWrapper(headers=headers)
requests_tools = [
    RequestsGetTool(requests_wrapper=requests_wrapper, response_length=3000),
    RequestsPostTool(requests_wrapper=requests_wrapper, response_length=3000),
]

In [32]:
# https://github.com/hwchase17/langchain/blob/7eba828e1b9542de4a0b67d1da2ffcfa83c6baca/langchain/agents/mrkl/base.py#L93-L94
# Prefix -> agent task description

# Only use information returned by the tools to construct your response.
# -> knows about some of these apis...
PREFIX="""You are an agent that interacts with a web API on behalf of a user.
If the user asks you a question or assigns you a task that seems unrelated to the API, do not hallucinate anything. Just say you don't know.
Only use information returned by the tools to construct your response.
"""
# Suffix -> warm-start the completion.
SUFFIX = """Begin!

Question: {input}
Thought: I should use /search endpoint to find Spotify resource ids.
{agent_scratchpad}"""

# describes LM context format i.e. how to construct a ReAct agent trajectory
FORMAT_INSTRUCTIONS

tools = [endpoint_retriever_agent_tool, *requests_tools]

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=PREFIX,
    suffix=SUFFIX,
    format_instructions=FORMAT_INSTRUCTIONS,
)


In [382]:
# print(prompt.template)

In [33]:
llm_chain = LLMChain(
    llm=OpenAI(model_name='text-davinci-003', temperature=0.0),
    prompt=prompt,
)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

In [34]:
# agent_executor.run("What is the spotify id for the song 'So What'?")
query = "What's the spotify id for the song 'So What'"
query = "Uses Spotify's recommend api to find songs by artists like Alicia Keys. Give me actual song names."
out = agent_executor({'input': query})



> Entering new AgentExecutor chain...
Action: endpoint_selector
Action Input: "Spotify recommend api"

> Entering new AgentExecutor chain...
Action: api_endpoint_retriever
Action Input: Spotify recommend
Observation: GET /artists/{id}/related-artists:
  description: 'Get Spotify catalog information about artists similar to a given artist.
    Similarity is based on analysis of the Spotify community''s [listening history](http://news.spotify.com/se/2010/02/03/related-artists/).

    '
  parameters: []
  responses:
    $ref: '#/components/responses/ManyArtists'
GET /me/albums:
  description: 'Get a list of the albums saved in the current Spotify user''s ''Your
    Music'' library.

    '
  parameters: []
  responses:
    $ref: '#/components/responses/PagingSavedAlbumObject'
GET /me/tracks:
  description: 'Get a list of the songs saved in the current Spotify user''s ''Your
    Music'' library.

    '
  parameters: []
  responses:
    $ref: '#/components/responses/PagingSavedTrackObject'

Thought: I should use the /recommendations endpoint to get the songs by artists like Alicia Keys.
Action: requests_get
Action Input: "https://api.spotify.com/v1/recommendations?seed_artists=3DiDSECUqqY1AuBP8qtaIa"
Observation: {
  "tracks" : [ {
    "album" : {
      "album_group" : "SINGLE",
      "album_type" : "SINGLE",
      "artists" : [ {
        "external_urls" : {
          "spotify" : "https://open.spotify.com/artist/0ZrpamOxcZybMHGg1AYtHP"
        },
        "href" : "https://api.spotify.com/v1/artists/0ZrpamOxcZybMHGg1AYtHP",
        "id" : "0ZrpamOxcZybMHGg1AYtHP",
        "name" : "Robin Thicke",
        "type" : "artist",
        "uri" : "spotify:artist:0ZrpamOxcZybMHGg1AYtHP"
      } ],
      "available_markets" : [ "AD", "AE", "AG", "AL", "AM", "AO", "AR", "AT", "AU", "AZ", "BA", "BB", "BD", "BE", "BF", "BG", "BH", "BI", "BJ", "BN", "BO", "BR", "BS", "BT", "BW", "BY", "BZ", "CA", "CD", "CG", "CH", "CI", "CL", "CM", "CO", "CR", "CV", "CW", "CY", "CZ", "DE", "DJ", "DK", "

Note, hallucinations in Final Answer... there's pretty minimal prompt engineering here atm tho.

In [ ]:
# Single-step: should call /search
# Demonstrative (no user would ask for a spotify id)
query = "What's the spotify id for the song 'So What'"
desired_output = '4sb0eMpDn3upAFfyi4q2rw'
out = agent_executor({'input': query})

def _retrieves_search_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'api_endpoint_retriever':
            if '/search' in observation:
                return True
    return False

def _calls_search_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'requests_get' and '/search' in action.tool_input:
            return True
    return False

def _check_answer(agent_out):
    output = agent_out['output']
    if desired_output in output:
        return True
    return False

tests = [_retrieves_search_endpoint, _calls_search_endpoint, _check_answer]
for test_fn in tests:
    print(test_fn.__name__, test_fn(out))

In [ ]:
# Single-step: should call /track or /tracks
# Demonstrative (no user would have a spotify id)
query = "What's the track name for the song with id 4vLYewWIvqHfKtJDk8c8tq?"
out = agent_executor({'input': query})
desired_output = "So What"

def _retrieves_tracks_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'api_endpoint_retriever':
            if '/track' in observation:
                return True
    return False

def _calls_tracks_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'requests_get' and '/track' in action.tool_input:
            return True
    return False

def _check_answer(agent_out):
    final_obs = agent_out['intermediate_steps'][-1][1]
    output = agent_out['output']
    # Verify the answer comes from context...    
    if desired_output in final_obs and desired_output in output:
        return True
    return False

tests = [_retrieves_tracks_endpoint, _calls_tracks_endpoint, _check_answer]
for test_fn in tests:
    print(test_fn.__name__, test_fn(out))

In [ ]:
# Single-step: calls /search
# Slightly harder
query = "What artist wrote the song So What"
out = agent_executor({'input': query})
desired_output = "Miles Davis"

def _retrieves_search_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'api_endpoint_retriever':
            if '/search' in observation:
                return True
    return False

def _calls_search_endpoint(agent_out):
    steps = agent_out['intermediate_steps']
    for step in steps:
        action, observation = step
        if action.tool == 'requests_get' and '/search' in action.tool_input:
            return True
    return False

def _check_answer(agent_out):
    final_obs = agent_out['intermediate_steps'][-1][1]
    output = agent_out['output']
    # Verify the answer comes from context...    
    if desired_output in final_obs and desired_output in output:
        return True
    return False

tests = [_retrieves_search_endpoint, _calls_search_endpoint, _check_answer]
for test_fn in tests:
    print(test_fn.__name__, test_fn(out))

#### The JSON explorer agent

In [ ]:

from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import RequestsWrapper
from langchain.tools.json.tool import JsonSpec

json_spec=JsonSpec(dict_=spec, max_value_length=4000)
requests_wrapper=RequestsWrapper(headers=headers)
openapi_toolkit = OpenAPIToolkit.from_llm(OpenAI(temperature=0), json_spec, requests_wrapper, verbose=True)

json_explorer_agent = create_openapi_agent(
    llm=OpenAI(temperature=0),
    toolkit=openapi_toolkit,
    verbose=True
)
json_explorer_agent.run("What is the spotify id for the song 'So What'?")